In [1]:
import requests
from pathlib import Path
import json
import pandas as pd
import datetime
from datetime import datetime, timezone
import hashlib
from sqlalchemy import create_engine
import numpy as np
from time import sleep
import logging
import pytz
import math
import os

In [2]:
from dotenv import load_dotenv
env_file = Path("../.env")
print(".env exists:", env_file.exists(), "→", env_file)
# Загружаем .env
loaded = load_dotenv(dotenv_path=env_file)

.env exists: True → ..\.env


In [3]:
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', 1000)  # Увеличить ширину отображения

In [4]:
# Текущее время
current_time = datetime.now(timezone.utc).isoformat()
current_time

'2025-12-08T11:26:27.363402+00:00'

In [6]:
url = os.getenv('URL')
headers = {
    'Content-Type': os.getenv('CONTENT_TYPE'),
    'Authorization': os.getenv('AUTHORIZATION'),
}
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

**Обновление пары Access Token и Refresh Token**

In [ ]:
url_ref = "https://demo-hs.fgis-saturn.ru/backend/role/newaccesstoken"

payload = {}
headers = {
  'refresh_token': REFRESH_TOKEN 
}

# Отправляя запрос, нужно понимать что пару токенов прийдется обновить

#response = requests.request("GET", url_ref, headers=headers, data=payload)

#print(response.text)

**Проверка регистрации компании в ФГИС САТУРН**

In [ ]:
payload = json.dumps({
  "com": "execOperation",
  "op": "static/getList()",
  "otype": "Contractor",
  "opargs":{
    "pos": 0,
    "size": 10,
    "getFullCards": 1,
    "filters": [
      {
        "column": "inn",
        "condition": "=",
        "value": [5029069967] # ИНН проверяемой компании
      },
      { "column": "lcState",
       "condition": "=", 
       "value": ["actual"] 
      }
    ]
  }
})

response = requests.post(url, headers=headers, data=payload)

#print(response.status_code)

registr_data = response.json()

In [9]:
# Пример ответа от САТУРН в формате HRJSON
registr_data

{'transId': None,
 'resCode': 200,
 'reqPerMin': None,
 'resMsg': None,
 'resDescription': None,
 'resData': {'objList': {'_FORMAT_VER': '2021.09.25_0430',
   '_rectype': 'array',
   '_OBJ_ARRAY': [{'id': '248824',
     'name': 'ООО "ЛЕ МОНЛИД"',
     'INN': '5029069967',
     'lcState': 'actual',
     'personFio': None,
     'legalAddress': '141031, Российская Федерация, Московская обл., г. Мытищи, Осташковское ш., д. 1',
     'factualAddress': '',
     'vetis_region': None,
     'responsiblePerson': 'ДЕФАССЬЕ ЛОРАН, ЛУИ, КЛОД ;ГЕНЕРАЛЬНЫЙ ДИРЕКТОР',
     'phoneNumber': '',
     'email': '',
     'KPP': '502901001',
     'OGRN': '1035005516105',
     'dateRegistration': 1055894400.0,
     'dateModified': '2025-03-16T23:01:17.564097458Z',
     'vetis_versionUUID': 'd6638f09-c355-45d1-9e13-f4dec68f774c',
     'vetis_type': 1,
     'vetis_incorporationForm': 'Имя: Общества с ограниченной ответственностью Код: 12300 Короткое имя: null',
     'fullName': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕН

In [10]:
# Пример работы с HRJSON как с обычным JSON

# Извлекаем нужные массивы
obj_array = registr_data["resData"]["objList"]["_OBJ_ARRAY"]

# Проверяем, что есть хотя бы один объект в массиве
if obj_array:
    obj = obj_array[0]
    
    # Выводим необходимые поля
    print("ID компании:", obj["id"])
    print("Наименование компании:", obj["name"])
    print("ИНН:", obj["INN"])
    print("Статус:", obj["lcState"])
    print("Руководитель:", obj["responsiblePerson"])
    print("Юридический адрес:", obj["legalAddress"])
else:
    print("Компания не зарегестрирована во ФГИС САТУРН")

ID компании: 248824
Наименование компании: ООО "ЛЕ МОНЛИД"
ИНН: 5029069967
Статус: actual
Руководитель: ДЕФАССЬЕ ЛОРАН, ЛУИ, КЛОД ;ГЕНЕРАЛЬНЫЙ ДИРЕКТОР
Юридический адрес: 141031, Российская Федерация, Московская обл., г. Мытищи, Осташковское ш., д. 1


**Прототип функции для первичной инвентаризации**

In [11]:
def get_description():
    while True:
        description = input("Введите описание документа (planned/unplanned): ").strip().lower()
        if description in ["planned", "unplanned"]:
            return description
        else:
            print("Неверный ввод. Пожалуйста, введите 'planned' или 'unplanned'.")

def process_inventory_data(inventory_data, url, headers):
    # Получаем текущее время в UTC и форматируем его
    now = datetime.now(timezone.utc)
    formatted_time = now.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

    # Запрашиваем ввод данных у пользователя
    name = input("Введите имя документа: ")  # Пример: "Тест ввода остатков от 2025-04-14"
    description = get_description()  # Используем функцию для выбора описания
    doc_num = input("Введите номер документа: ")  # Пример: "Test_Nick_Array_002"
    warehouse_id = int(input("Введите ID склада: "))  # Пример: 1077174
    reason = input("Введите причину: ")  # Пример: "проверка базового скрипта_2"

    # Формируем batch_list и pat_dto_list из DataFrame
    batch_list = []
    pat_dto_list = []
    
    for index, row in inventory_data.iterrows():
        batch_info = {
            "patProductId": row['patProductId'],
            "baseUnitType": row['baseUnitType'],
            "puKgWeightFact": row['puKgWeightFact'],
            "puUnitFact": row['puUnitFact'],
            "batchCode": row['batchCode'],
            "batchId": row['batchId'],
            "expirationDate": row['expirationDate'],
            "countPuFact": row['countPuFact'],
            "countGapStatus": "Первичный ввод остатков"
        }
        batch_list.append(batch_info)
        
        pat_dto_info = {
            "patProductId": row['patProductId'],
            "batchId": row['batchId'],
            "countPuFact": row['countPuFact'],
            "countGapStatus": "Первичный ввод остатков"
        }
        pat_dto_list.append(pat_dto_info)

    # Формирование первого payload для создания нового черновика
    payload = json.dumps({
        "otype": "Inventorization",
        "com": "execOperation",
        "op": "static/createNew()",
        "opargs": {
            "theCard": {
                "head": {
                    "name": name,
                    "description": description,
                    "docNum": doc_num,
                    "docDate": formatted_time,
                    "warehouseId": warehouse_id,
                    "reason": reason
                },
                "batchList": batch_list
            }
        }
    })

    # Первый запрос на создание черновика
    response = requests.post(url, headers=headers, data=payload)
    
    if response.status_code == 200:
        print("Черновик Акта создан успешно!")
    else:
        print("Обработано с ошибкой:", response.status_code)
    #print("Создание черновика:", response.status_code)
    
    inventory_data_response = response.json()
    in_id = inventory_data_response['resData']['id']

    # Формирование второго payload для перевода черновика в актуализированный статус
    payload = json.dumps({
        "otype": "Inventorization",
        "com": "execOperation",
        "op": "draft/setActual()",
        "oid": in_id,  # id полученное при создании черновика
        "opargs": {
            "theCard": {
                "head": {
                    "id": in_id,
                    "name": name,
                    "description": description,
                    "docNum": doc_num,
                    "docDate": formatted_time,
                    "warehouseId": warehouse_id,
                    "reason": reason  
                },
                "patDtoList": pat_dto_list
            }
        }
    })

     # Второй запрос на обновление черновика
    response = requests.post(url, headers=headers, data=payload)

    if response.status_code == 200:
        print("Актуализировано успешно, проверяйте в интерфейсе Акт:", doc_num)
    else:
        print("Обработано с ошибкой:", response.status_code)
    
    #print("Обновление черновика:", response.status_code)
    #print(response.json)

In [12]:
inventory_data = {
    'patProductId': [3922, 3203, 3106, 3770, 3163, 5538],
    "baseUnitType" : ["кг", "кг", "кг", "кг", "кг", "кг"],
    "puKgWeightFact" : [1, 1, 1, 1, 1, 1],
    "puUnitFact" : ["Упаковка", "Упаковка", "Упаковка", "Упаковка", "Упаковка", "Упаковка"],
    "expirationDate": ["2026-04-30T21:00:00.001Z", "2026-04-30T21:00:00.001Z", "2026-04-30T21:00:00.001Z", "2026-04-30T21:00:00.001Z", "2026-04-30T21:00:00.001Z", "2026-04-30T21:00:00.001Z"],
    "countPuFact" : [11, 11, 11, 11, 11, 11],
    "batchCode" : ["1234", "3245", "1445", "6789", "9876", "5423"],
    "batchId" : [0, 0, 0, 0, 0, 0]
    
}

inventory_data = pd.DataFrame(inventory_data)

display(inventory_data)

,patProductId,baseUnitType,puKgWeightFact,puUnitFact,expirationDate,countPuFact,batchCode,batchId
0,3922,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,1234,0
1,3203,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,3245,0
2,3106,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,1445,0
3,3770,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,6789,0
4,3163,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,9876,0
5,5538,кг,1,Упаковка,2026-04-30T21:00:00.001Z,11,5423,0


In [13]:
process_inventory_data(inventory_data, url, headers)

Неверный ввод. Пожалуйста, введите 'planned' или 'unplanned'.
Неверный ввод. Пожалуйста, введите 'planned' или 'unplanned'.
Черновик Акта создан успешно!
Актуализировано успешно, проверяйте в интерфейсе Акт: 560456
